In [1]:
from math import floor 
import pandas as pd

def estimate_n_fft(f_max, sampling_rate):
    # Calculate the frequency resolution needed to capture up to f_max
    df = []
    for n_fft in range(2, 512+1, 2):
        bins = (n_fft // 2) + 1
        delta_f = sampling_rate / n_fft
        used_bins = 1 + floor(f_max / delta_f)
        used_perc = used_bins/bins
        df.append({"n_fft": n_fft, "used_perc": used_perc})
    df = pd.DataFrame(df).sort_values(by="used_perc", ascending=False)
    return df
    # # Estimate n_fft based on the desired frequency resolution
    # n_fft = int(sampling_rate / delta_f)
    
    # # Return the nearest even value to ensure the Nyquist frequency is captured
    # return n_fft + (n_fft % 2)

# Test
f_s = 128  # Example sampling rate
f_max = 50  # Example max frequency
estimate_n_fft(f_max, f_s)  # This should give an estimate for n_fft


,n_fft,used_perc
3,8,0.800000
8,18,0.800000
17,36,0.789474
31,64,0.787879
12,26,0.785714
...,...,...
9,20,0.727273
5,12,0.714286
1,4,0.666667
4,10,0.666667


In [3]:
import torch
import torchaudio.transforms as T

# Define a simple "network" using the MelScale and InverseMelScale transforms
class TestMelAndInverseMelScale(torch.nn.Module):
    def __init__(self, n_mels=40, sample_rate=16000, n_fft=400):
        super(TestMelAndInverseMelScale, self).__init__()
        self.mel_scale = T.MelScale(n_mels=n_mels, sample_rate=sample_rate, n_stft=n_fft//2+1)
        self.inverse_mel_scale = T.InverseMelScale(n_stft=n_fft//2+1, n_mels=n_mels, sample_rate=sample_rate)

    def forward(self, specgram):
        mel_output = self.mel_scale(specgram)
        o = self.inverse_mel_scale(mel_output)
        return o
    
# Create a dummy spectrogram
specgram = torch.randn(1, 201, 100, requires_grad=True)  # 1 channel, 201 freq bins, 100 frames

# Pass through the network
model = TestMelAndInverseMelScale()
output = model(specgram)

# Compute a dummy loss
loss = output.mean()

# Backward pass
loss.backward()

print("Backpropagation succeeded!")


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.